Authors: Salvatore Mandra (salvatore.mandra@nasa.gov)<br>
&emsp;&emsp;&emsp;&emsp;Jeffrey Marshall (jeffrey.s.marshall@nasa.gov)

Copyright © 2021, United States Government, as represented by the Administrator
of the National Aeronautics and Space Administration. All rights reserved.

The *HybridQ: A Hybrid Simulator for Quantum Circuits* platform is licensed under
the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed
under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
CONDITIONS OF ANY KIND, either express or implied. See the License for the
specific language governing permissions and limitations under the License.

# 01 - Gates

Gates are at the core of **HybridQ**. In this tutorial, we will show how gates are designed, and how to use them. Also, we will show how to create and implement new gates in **HybridQ**.

In **HybridQ** we use the concept of *properties* to define gates. More precisely, instead of using monolithic gates which are hard to maintain and extend, *every* property is independently implemented so that different object may share multiple properties. This approach not only simplifies the development by allowing a large reuse of the code, it also simplify the maintenance and extension of the code as new features and fix are applied at the level of the properties, and then propagated to all objects using them.

For instance, the rotation gate on the <code>X</code> axis inherits the following properties:

In [1]:
%load_ext autoreload
%autoreload 2

# Import NumPy
import numpy as np

# Load gates
from hybridq.gate import Gate

# Get inheritance from rotation gate
type(Gate('RX')).mro()

[hybridq.base.base.Gate_RX,
 hybridq.gate.gate.BaseGate,
 hybridq.gate.property.RotationGate,
 hybridq.gate.property.ParamGate,
 hybridq.base.property.Params,
 hybridq.gate.property.UnitaryGate,
 hybridq.gate.property.PowerMatrixGate,
 hybridq.gate.property.PowerGate,
 hybridq.gate.property.QubitGate,
 hybridq.base.property.Tags,
 hybridq.base.property.Name,
 hybridq.base.base.__Base__,
 object]

Every single property add (or modify) some characteristic of the gate. For instance, <code>PowerGate</code> allows to specify any power to the given gate while <code>PowerMatrixGate</code> takes a matrix representing the gate and actually implement the operation.

For the full set of properties that each gate supports, please refer to the technical [documentation](https://github.com/nasa/hybridq/tree/main/docs).

## Gates

At the moment, **HybridQ** supports two types of gates: *named* gates and *functional* gates. Named gates, as the name suggests, are the commonly used gates while functional gates allow to interact with the quantum state in a non-trivial way.

### Named Gates

A full list of available named gates can be obtained as follows:

In [2]:
from hybridq.gate.utils import get_available_gates

get_available_gates()

('I',
 'H',
 'X',
 'Y',
 'Z',
 'U3',
 'R_PI_2',
 'ZZ',
 'CZ',
 'CX',
 'SWAP',
 'ISWAP',
 'CPHASE',
 'FSIM',
 'RX',
 'RY',
 'RZ',
 'SQRT_X',
 'SQRT_Y',
 'P',
 'T',
 'SQRT_SWAP',
 'SQRT_ISWAP')

In [3]:
# All named gates can be created by using the following syntax
# (the name of the gate is case insensitive):
Gate('iswap')

Gate_ISWAP(name='ISWAP', n_qubits=2, M=numpy.ndarray(shape=(4, 4), dtype=complex128))

Some gates require parameters to specify details on the gate. For example, the rx gate expects one parameter, the rotation angle.

HybridQ provides an easy way to determine the number of qubits and parameters required to specify the gate

In [4]:
for name in ['rx', 'fsim', 'x']:
    g = Gate(name)
    print(f"{name.upper():5s} - Number of qubits:", g.n_qubits)
    print(f"{'':5s} - Number of params:",
          g.n_params if g.provides('n_params') else 'none')

RX    - Number of qubits: 1
      - Number of params: 1
FSIM  - Number of qubits: 2
      - Number of params: 2
X     - Number of qubits: 1
      - Number of params: none


Qubits and parameters can be specified either before or after the creation of the gate. **HybridQ** allows the use of *any* object for qubits and parameters, including <code>sympy</code> symbols.

In [5]:
# Example 1: Symbolic qubits and params

from sympy import symbols

# Define a few symbols
α, β, γ = symbols('α β γ')

# Specify all parameters at the creation of the gate
g1 = Gate('RX', qubits=[α], params=[β], power=γ)

print(g1)

Gate_RX(name='RX', qubits=(α,), φ=Mod(1.0*β*γ, 12.5663706143592))


In [6]:
# Example 2: Specify inplace after creation

# Get a rotation gate without qubits nor parameters
g2 = Gate('RX')

# Add some qubits inplace
g2.on([α], inplace=True)

# Set parameters as new object and elevate to the power of γ
g2 = g2.set_params([β])**γ

# Check g1 == g2
assert (g1 == g2)

print(g2)

Gate_RX(name='RX', qubits=(α,), φ=Mod(1.0*β*γ, 12.5663706143592))


In [7]:
# Because Gate('RX') is a 'RotationGate', raising it to any
# power will actually change the rotation angle
assert (g1.power == 1)
assert (g1.params[0] == (β * γ) % (4 * np.pi))

All gates provide a way to *tag* them. This is particularly useful when one wants to identify some specific gates in a circuit.

In [8]:
g1 = g1.set_tags(dict(a=1, b=2))
g2.set_tags(dict(a=2), inplace=True)

# Tags are not used when gates are compared
assert (g1 == g2)

g1, g2, g1.tags['a'], g2.tags['a']

(Gate_RX(name='RX', qubits=(α,), φ=Mod(1.0*β*γ, 12.5663706143592), tags={'a': 1, 'b': 2}),
 Gate_RX(name='RX', qubits=(α,), φ=Mod(1.0*β*γ, 12.5663706143592), tags={'a': 2}),
 1,
 2)

It is possible to get the matrix representing the gate by using the following command:

In [9]:
try:
    # This should fail because no values have been provided to symbols
    g1.matrix()
except:
    pass
else:
    raise RuntimeError("Something went wrong")

# Let's specify the rotation angle
g1.set_params([g1.params[0].subs({β: 1.1, γ: 2.2})], inplace=True).matrix()

array([[0.3530194+0.j      , 0.       -0.935616j],
       [0.       -0.935616j, 0.3530194+0.j      ]])

Gates can be also created by providing their matrix representation using the named gate *MatrixGate*

In [10]:
from hybridq.gate import MatrixGate

# Create a gate using its matrix representation
gm = MatrixGate(qubits=g1.qubits, U=g1.matrix())

# It is also possible to use Gate directly
gm2 = Gate('matrix', qubits=g1.qubits, U=g1.matrix())
assert (gm2 == gm)

# Observe that the strict equivalence '==' requires that the
# two objects are identical (including qubits and parameters)
assert (gm != g1)

# However, HybridQ provides a way to compare two gates
assert (gm.isclose(g1))

# 'isclose' will return False if qubits are different
assert (not gm.isclose(g1.on([42])))

In [11]:
# The order of qubits is important. For instance the following
# CNOT gate, will flip the qubit 'b' only if 'a' is one
g = Gate('cnot', qubits=['a', 'b'])

# The matrix representation assumes that the first qubit
# is 'a' and the second qubit is 'b'
g.matrix()

/home/dagart/local/miniconda3/envs/hybridq/lib/python3.8/site-packages/hybridq/gate/gate.py:291: UserWarning: 'CNOT' is an alias for 'CX'. Using Gate(name='{_gate_aliases[name]}').
  warn(f"'{name}' is an alias for '{_gate_aliases[name]}'. "


array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])

In [12]:
# If a different order is required, it must be specified
g.matrix(order=['b', 'a'])

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

**HybridQ** also allows users to split any gate in terms of its Schmidt decomposition. More precisely, given any gate $G$ acting on two subset of qubits, it is possible to write $G$ as:
$$
    G = \sum_i s_i\, G^{(1)}_i \otimes G^{(2)}_i
$$
with $s_i$ being real and positive weights, and $G_i^{(1)}$ and $G_i^{(2)}$ being gates acting only on either the first or the second subset of qubits respectively.

In [13]:
# Load the gate utils
from hybridq.gate import utils as gate_utils

# Get gate
g = Gate('FSIM', qubits=[1, 2], params=[1.1, 2.2])**3.3

# Return the Schmidt decomposition of gate
s = gate_utils.decompose(g, qubits=[1])

# Print weights
print('Weights:', s.s)

# Print "left" gates
print('Left gates:')
for _g in s.gates[0]:
    print('\t', _g)

# Print "right" gates
print('\nRight gates:')
for _g in s.gates[0]:
    print('\t', _g)

# Gate can be easily recomposed using the complementary function
g_merged = MatrixGate(sum(s * gate_utils.merge(g1, g2).matrix(order=g.qubits)
                          for s, g1, g2 in zip(s.s, *s.gates)),
                      qubits=g.qubits)

# Check
assert (g.isclose(g_merged))

Weights: [1.82742925 0.46922004 0.46922004 0.46922004]
Left gates:
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))

Right gates:
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))
	 MatrixGate(name='MATRIX', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=complex128))


In [14]:
# HybridQ also provides a way to check if two gates commute
g1 = Gate('CX', [0, 1])
g2 = Gate('X', [1])

# Since the 'X' gate is acting on the controlled qubit, g1 and g2 should commute
assert (g1.commutes_with(g2))

# However, it won't commute if X would act on the controlling qubit
assert (not g1.commutes_with(g2.on([0])))

# Gates not sharing any qubits always commute
assert (g1.commutes_with(Gate('U3', qubits=[3], params=[1, 2, 3])))

# Commutation is only available for gate with qubits
try:
    g1.commutes_with(Gate('X'))
except Exception as e:
    print(e)
else:
    raise RuntimeError("Something went wrong")

Cannot check commutation between virtual gates.


Gates can be conjugated and transposed:

In [15]:
# 
g = Gate('FSIM', params=[1, 2])
gt = g.T()
gc = g.conj()
ga = g.adj()

print(gt)
print(gc)
print(ga)

assert (gt.is_transposed())
assert (gc.is_conjugated())
assert (ga.is_conjugated())
assert (ga.is_transposed())

# Checks
np.testing.assert_allclose(g.matrix().T, gt.matrix())
np.testing.assert_allclose(g.matrix().conj(), gc.matrix())
np.testing.assert_allclose(g.matrix().conj().T, ga.matrix())

Gate_FSIM^T(name='FSIM', n_qubits=2, params=(1, 2))
Gate_FSIM^*(name='FSIM', n_qubits=2, params=(1, 2))
Gate_FSIM^+(name='FSIM', n_qubits=2, params=(1, 2))


**HybridQ** supports self-adjoint gates

In [16]:
g = Gate('Y')

# Print gates
print(g)
print(g.conj())
print(g.T())
print(g.conj().T())

# Check
assert (g.conj().T() == g)
assert (not g.conj() == g)
assert (not g.T() == g)
assert (g.adj() == g)

# This also apply for the inverse
assert (g.inv() == g)
assert (g.adj() == g)

# This is not true anymore if any power different from 1 or -1 is applied
print((g**1.1).adj())

# Check
assert (g**-1 == g)
assert ((g**1.1).adj() != g)

Gate_Y(name='Y', n_qubits=1, M=numpy.ndarray(shape=(2, 2), dtype=complex128))
Gate_Y^*(name='Y', n_qubits=1, M=numpy.ndarray(shape=(2, 2), dtype=complex128))
Gate_Y^T(name='Y', n_qubits=1, M=numpy.ndarray(shape=(2, 2), dtype=complex128))
Gate_Y(name='Y', n_qubits=1, M=numpy.ndarray(shape=(2, 2), dtype=complex128))
Gate_Y^+(name='Y', n_qubits=1, M=numpy.ndarray(shape=(2, 2), dtype=complex128))**1.1


### Functional Gate

**HybridQ** supports different functional gates that allow a great control on the quantum circuit simulation. At the moment, **HybridQ** supports the following functional gates:

<od>
    <li><code>Control</code>
    <li><code>TupleGate</code>
    <li><code>SchmidtGate</code>
    <li><code>StochasticGate</code>
    <li><code>Projection</code>
    <li><code>Measure</code>
    <li><code>FunctionalGate</code>
</od>

#### <code>Control</code>

As the name suggests, this functional gate allows to create any controlled gate. Controlled gates extend any gate to only activate when all of the control qbits are |1>.

In [17]:
# Import control gate
from hybridq.gate import Control

# Let's create a random gate
g = MatrixGate(np.random.random((2**4, 2**4)), qubits='abcd')

# Create a control gate with 3 controlling qubits
cg = Control(c_qubits=[('x', 1), ('y', 2), ('z', 3)], gate=g)
# g is provided as the gate to be controlled.
# Now the g gate will only be applied when the 3 controlling qubits are all |1>

# Print gate
print(cg)

# Initialize matrix with identity
U = np.reshape(np.eye(2**cg.n_qubits),
               (2, ) * 3 + (2**4, ) + (2, ) * 3 + (2**4, ))

# Apply g.matrix only when all controlling qubits are 1
U[(1, 1, 1, slice(2**4), 1, 1, 1, slice(2**4))] = g.matrix()

# Reshape to the right shape
U = np.reshape(U, (2**7, 2**7))

# Check
np.testing.assert_allclose(U, cg.matrix())

Controlled_MatrixGate(name='CCCMATRIX', n_qubits=7, c_qubits=(('x', 1), ('y', 2), ('z', 3)), qubits=(('x', 1), ('y', 2), ..., c, d), gate=MatrixGate(name='MATRIX', qubits=('a', 'b', 'c', 'd'), M=numpy.ndarray(shape=(16, 16), dtype=float64)))


#### <code>TupleGate</code>

<code>TupleGate</code> gathers together multiple gates. The combined TupleGate can then be treated as a single gate. For example,

In [18]:
# Import TupleGate
from hybridq.gate import TupleGate
from hybridq.utils import sort

# Let's define some gates ...
g1 = Gate('X')
g2 = Gate('Y', qubits='a')
g3 = Gate('swap', qubits='ab')

# ... and create a TupleGate
tg = TupleGate((g1, g2, g3))

# If some gates do not have qubits, both qubits and n_qubits are None
assert (tg.qubits is None and tg.n_qubits is None)

# If every gate has qubits, when TupleGate will have
# all qubits (counted once) from every gate
tg[0].on([1], inplace=True)

# For consistency, qubits are always sorted using 'hybridq.utils.sort'
print(f'Qubits: {tg.qubits}')
print(f'Number of qubits: {tg.n_qubits}')

Qubits: (1, 'a', 'b')
Number of qubits: 3


Once a tuple is created, you can add additional gates using the addition operator (i.e., '+'),

In [19]:
# It is possible to add gates ...
tg = tg + (g2, g3)

# ... and one can concatenate multiple tuples
tg = tg + (tg, tg)
tg = tg + (tg, tg)

# Check that gates and number of gates haven't changed
assert (tg.n_qubits == 3 and tg.qubits == tuple(sort(['a', 'b', 1])))

 The TupleGate also provides a way to flatten the collection

In [20]:
print(tg.flatten())

TupleGate(name='TUPLE', Gate_X(name='X', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=int64)), Gate_Y(name='Y', qubits=('a',), M=numpy.ndarray(shape=(2, 2), dtype=complex128)), Gate_SWAP(name='SWAP', qubits=('a', 'b'), M=numpy.ndarray(shape=(4, 4), dtype=int64)), Gate_Y(name='Y', qubits=('a',), M=numpy.ndarray(shape=(2, 2), dtype=complex128)), Gate_SWAP(name='SWAP', qubits=('a', 'b'), M=numpy.ndarray(shape=(4, 4), dtype=int64)), Gate_X(name='X', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=int64)), Gate_Y(name='Y', qubits=('a',), M=numpy.ndarray(shape=(2, 2), dtype=complex128)), Gate_SWAP(name='SWAP', qubits=('a', 'b'), M=numpy.ndarray(shape=(4, 4), dtype=int64)), Gate_Y(name='Y', qubits=('a',), M=numpy.ndarray(shape=(2, 2), dtype=complex128)), Gate_SWAP(name='SWAP', qubits=('a', 'b'), M=numpy.ndarray(shape=(4, 4), dtype=int64)), Gate_X(name='X', qubits=(1,), M=numpy.ndarray(shape=(2, 2), dtype=int64)), Gate_Y(name='Y', qubits=('a',), M=numpy.ndarray(shape=(2, 2), dtype=complex

#### <code>SchmidtGate</code>

As the name suggest, a <code>SchmidtGate</code> allow to create a gate from its Schmidt decomposition. More precisely, it allows to define gates as:
$$
S = \sum_{ij} s_{ij}\,G^{(1)}_i \otimes G^{(2)}_j
$$
with $s_{ij}$ being an arbitrary matrix of real and positive weights, and $G^{(1)}_i$ and $G^{(2)}_j$ are gates acting on two different set of qubits. The <code>SchmidtGate</code> is also the gate returned by <code>hybridq.gate.utils.decompose</code>.

In [21]:
# Import SchmidtGate
from hybridq.gate import SchmidtGate, MatrixGate

# Let's create some random gates ...
G1 = [
    MatrixGate(np.random.random((4, 4)), qubits='ab'),
    MatrixGate(np.random.random((4, 4)), qubits='cb'),
    MatrixGate(np.random.random((2, 2)), qubits='b'),
]
G2 = [
    MatrixGate(np.random.random((4, 4)), qubits=[2, 3]),
    MatrixGate(np.random.random((4, 4)), qubits=[5, 3]),
    MatrixGate(np.random.random((4, 4)), qubits=[(0, 1), 2]),
    MatrixGate(np.random.random((2, 2)), qubits=[2]),
]

# ... and random weights
w = np.random.random((3, 4))

# Create SchmidtGate
S = SchmidtGate(gates=(G1, G2), s=w)

#assert (type(gate_utils.decompose(Gate('X', [1]), [1])) == type(S))

# Qubits in SchmidtGate's are ordered so that "left" qubits
# appear before than the "right" qubits
assert (S.qubits == S.gates[0].qubits + S.gates[1].qubits)

# Let's check the SchmidtGate by manually merging the gates
#
# First, we need to extend all gates in G1 and G2 to the right number of qubits, ...
_G1 = [gate_utils.merge(g, Gate('I', qubits=S.qubits)) for g in G1]
_G2 = [gate_utils.merge(g, Gate('I', qubits=S.qubits)) for g in G2]

# ... we can then sum all matrices using the right order ...
U = np.sum([
    S.s[i, j] * gate_utils.merge(_G1[i], _G2[j]).matrix(order=S.qubits)
    for i in range(len(G1)) for j in range(len(G2))
],
           axis=0)

# ... and check that everything is ok
np.testing.assert_allclose(U, S.matrix())

#### <code>StochasticGate</code>

<code>StochasticGate</code>'s are used to define a set of gates that are stochastically applied to a circuit with a given probability <code>p</code>. Let's consider the case of a simple depolarizing channel of the form:
$$
    \rho \rightarrow \mathcal{E}(\rho) = (1-p)\,\rho 
        + \frac{p}{3}\,\sigma_x \rho\, \sigma_z 
        + \frac{p}{3}\,\sigma_y \rho\, \sigma_z 
        + \frac{p}{3}\,\sigma_z \rho\, \sigma_z 
$$
with $\sigma_x$, $\sigma_y$ and $\sigma_z$ being the Pauli matrices. Since the noise channel is composed of unitary matrices, the evolution of the quantum state can be done by running the quantum circuit simulations multiple times, each time randomly sampling from the noise channel either the identity (with probability $1-p$), or the $\sigma_x$/$\sigma_y$/$\sigma_z$ gate (with probability $p/3$). The <code>StochasticGate</code> allows to easily represent such cases.

In [22]:
# Import StochasticGate
from hybridq.gate import StochasticGate
from collections import Counter

# Let's define the depolarizing probability
p = 0.42

# Define the four gates in the depolarizing channel
I, X, Y, Z = [Gate(g, [0]) for g in 'IXYZ']

# Define the stochastic gate
S = StochasticGate(gates=[I, X, Y, Z], p=[1 - p, p / 3, p / 3, p / 3])

# We can sample multiple times from S
print([g.name for g in S.sample(10)])

# Let's count the number of times the gate appear in the sample
# (it may take a while)
_count = Counter(g.name for g in S.sample(1000000))

# Check
assert (np.isclose(_count['I'] / 1000000, 1 - p, atol=1e-3))
assert (np.isclose(_count['X'] / 1000000, p / 3, atol=1e-3))
assert (np.isclose(_count['Y'] / 1000000, p / 3, atol=1e-3))
assert (np.isclose(_count['Z'] / 1000000, p / 3, atol=1e-3))

['Z', 'I', 'I', 'I', 'X', 'X', 'Y', 'Z', 'X', 'I']


#### <code>Projection</code> and <code>Measure</code>

The gate <code>Projection</code> can be used to project qubits to a specific state.

In [23]:
# Import Projection
from hybridq.gate import Projection

# Let's define a random quantum state of 5 qubits
psi = np.random.random((2, ) * 5) + 1j * np.random.random((2, ) * 5)
psi /= np.linalg.norm(psi.ravel())

# Define some qubits ...
qubits = np.random.randint(2**32, size=5)

# ... and the subset to project
p_qubits = np.random.choice(qubits, size=2, replace=False).tolist()

# Set the desired state to project the p_qubits
p_state = bin(np.random.randint(2**2))[2:].zfill(2)

# Create projection
P = Projection(state=p_state, qubits=p_qubits)

# Project quantum state
p_psi, _ = P(psi=psi, order=qubits)

# Let's check that the projection is correct
_psi = np.zeros_like(psi)
_proj = tuple(
    int(p_state[p_qubits.index(q)]) if q in p_qubits else slice(2)
    for q in qubits)
_psi[_proj] = psi[_proj]
_psi /= np.linalg.norm(_psi.ravel())

# Check
np.testing.assert_allclose(_psi, p_psi)

<code>Measure</code> gates is to <code>Projection</code> gates. However, unlike <code>Projection</code> gates, <code>Measure</code> gates project qubits to a given state with a probability given by the quantum state (that is, projection are sampled with the right quantum distribution).

In [24]:
# Import Measure
from hybridq.gate import Measure

# Let's define a random quantum state of 5 qubits
psi = np.random.random((2, ) * 5) + 1j * np.random.random((2, ) * 5)
psi /= np.linalg.norm(psi.ravel())

# Define some qubits ...
qubits = np.random.randint(2**32, size=5)

# ... and the subset two project
p_qubits = np.random.choice(qubits, size=2, replace=False).tolist()

# Set the desired state to project the p_qubits
p_state = bin(np.random.randint(2**2))[2:].zfill(2)

# Create measurement
M = Measure(qubits=p_qubits)

# Measure quantum state
p_psi, _ = M(psi=psi, order=qubits)

/home/dagart/local/miniconda3/envs/hybridq/lib/python3.8/site-packages/hybridq/utils/transpose.py:166: UserWarning: Fallback to 'numpy.transpose'
  warn("Fallback to 'numpy.transpose'")


#### <code>FunctionalGate</code>

<code>FunctionalGate</code>'s are the most general gate that are implementable in **HybridQ**. More precisely, <code>FunctionalGate</code>'s allow to directly manipulate the quantum state using a defined function to easily implement gates (such as oracle) that would be hard to implement otherwise.

In the following example, we will show how to implement a Grover oracle in **HybridQ**.

In [25]:
# Import FunctionalGate
from hybridq.gate import FunctionalGate, Projection

# Let's define the target state
target_state = bin(np.random.randint(2**3))[2:].zfill(3)


# FunctionalGate requires two objects: the function to apply
# to the quantum state and the qubits the gate will act on
#
# First, let's define the functional for the FunctionalGate
def grover(self, psi, order):
    """
    Given a quantum state ‘psi‘ with qubits given
    by ‘order‘, invert the phase of the state
    0....0.
    
    Parameters
    ----------
    psi:
        Because of internal optimizations, any functional
        for the 'FunctionalGate' must accept two different 
        format for 'psi': either an n-dimensional array of
        complex numbers (with 'n' the number of qubits) or
        a pair of n-dimensional arrays of real number, the
        first one representing the real part of the quantum
        state while the second one representing its imaginary
        part.
        
        In this case, 'Projection' is already able to distinguish
        between the two cases.
    order:
        It is the order of qubits used to represent 'psi'
        
    Returns
    -------
    new_psi, new_order
    """

    # Check
    assert (len(target_state) == self.n_qubits)

    # First, let’s create a projector
    proj = Projection(state=target_state, qubits=self.qubits)

    # Project quantum state
    psi0, new_order = proj(psi=psi, order=order, renormalize=False)

    # Check that order hasn’t changed
    assert (new_order == order)

    # Update quantum state
    psi -= 2 * psi0

    # Return quantum state
    return psi, order


# Then, one can create the Grover gate as follows
grover_gate = FunctionalGate(f=grover, n_qubits=3)

# Print
print(grover_gate)

# FunctionalGate's provide qubits and n_qubits, but not matrix
assert (grover_gate.provides('qubits,n_qubits'))
assert (not grover_gate.provides('matrix'))

# Define a superposition of three qubits
psi = np.ones(shape=(2, 2, 2), dtype='complex')
psi /= np.linalg.norm(psi)

# Apply Grover gate to state
new_psi, _ = grover_gate.on([0, 1, 2])(psi, order=[0, 1, 2])

# Print output (the state with '*' should have the sign inverted)
for i, x in enumerate(new_psi.ravel()):
    # Pad bitstring
    i = bin(i)[2:].zfill(3)

    # Print
    print('{0} {1}: {2:+g}'.format('*' if i == target_state else ' ', i,
                                   np.real_if_close(x)))

FunctionalGate(name='FUNCTIONAL', n_qubits=3)
  000: +0.353553
  001: +0.353553
  010: +0.353553
* 011: -0.353553
  100: +0.353553
  101: +0.353553
  110: +0.353553
  111: +0.353553


## Gate Implementation

Every property must inherit from <code>\_\_Base__</code>, which will add to the object the basic functionality to work with the **HybridQ** library.

In [26]:
from hybridq.base import __Base__


class A(__Base__):
    pass

The **HybridQ** library provides useful decorator to help developing new properties. At the moment, **HybridQ** provides:
<od>
    <li><code>staticvars</code>
    <li><code>compare</code>
    <li><code>requires</code>
    <li><code>generate</code>
</od>

<code>staticvars</code> is used to add variables which must provided while creating the object. <code>compare</code> is used to specify which variables should be compared in <code>\_\_eq__</code>. Similarly, <code>requires</code> is used to specify which variables the final object should have once created. Finally, <code>generate</code> is used to create **HybridQ** objects.

In [27]:
from hybridq.base import staticvars, generate


# Let's start creating an object with two static variables a and b,
# with a having a default value a=1. We want to make sure that a is
# always an integer between 0 and 10 while b can be any string.
@staticvars('a,b',
            a=1,
            transform=dict(a=int, b=str),
            check=dict(a=lambda x: 0 <= x <= 10))
class A(__Base__):
    pass


# Generate new type
new_type = generate('NewType', (A, ), b=2.2)

# Get new object
o = new_type()

# Get values
print((o.a, o.b))

# Trying to create a new type with a not in [0, 10] will fail
try:
    generate('NewType', (A, ), a=-1, b='this is a string')
except Exception as e:
    print(e)
else:
    raise RuntimeError("Something went wrong")

# Also, not specifying b will fail
try:
    generate('NewType', (A, ))
except Exception as e:
    print(e)
else:
    raise RuntimeError("Something went wrong")

# Static variables are by default read-only
try:
    o.a = 1
except Exception as e:
    print(e)
else:
    raise RuntimeError("Something went wrong")

(1, '2.2')
Check failed for 'a'
Static variable 'b' must be provided.
Cannot set static variable


In [28]:
from hybridq.base import compare

# By default, '__eq__' does not compare any variables
o1 = generate('NewType', (A, ), a=1, b=2)()
o2 = generate('NewType', (A, ), a=3, b='hello!')()

# Even if the two objects have different static vars,
# __eq__ returns True
assert (o1 == o2)


# The decorator 'compare' can be used to specify which
# variables to compare
@compare('a,c', cmp=dict(c=lambda x, y: int(x) == int(y)))
class B(A, b='default'):
    def __init_subclass__(cls, b=None, **kwargs):
        # If 'b' is not provided, use the default value
        b = cls.__get_staticvar__('b') if b is None else b

        # Call super
        super().__init_subclass__(b=b, **kwargs)

    def __init__(self, c):
        self.c = c


# Generate new objects
o1 = generate('NewTypeB', (B, ))(c=1)
o2 = generate('NewTypeB', (B, ))(c=2)
o3 = generate('NewTypeB', (B, ), b='not default')(c=1.3)

# Because a and c are compared, only o1 == o3
assert (o1 != o2)
assert (o1 == o3)
assert (o2 != o3)

In [29]:
from hybridq.base import requires


# Object may require variables that are not directly
# implemented but provided at the creation time
@requires('b')
class A(__Base__):
    pass


@staticvars('b')
class B(__Base__):
    pass


# Creating a new type with 'A' alone will fail
try:
    generate('NewType', (A, ))
except Exception as e:
    print(e)
else:
    raise RuntimeError("Something went wrong")

# However, since 'B' provides 'b', generating a
# new type from (B, A) will succede
generate('NewType', (B, A), b=42)

The following required attributes are not provided: ('b',)


hybridq.base.base.NewType

In [30]:
import pickle


# All objects derived from __Base__ are pickable. This
# is very useful when the use of 'multiprocessing' is
# required
@staticvars('a,b,c', a=1, b=2, c=3)
class A(__Base__):
    pass


@staticvars('x,y', x='a', y='b')
class B(A):
    def __init__(self, q=None):
        self.q = q


# Get new object
o = generate('NewType', (B, A))(q='test')

# Print
for k in 'abcxyq':
    print(f'o[{k}] = {getattr(o, k)}')

# Dump/load using pickle
o = pickle.loads(pickle.dumps(o))

# Print
print('---')
for k in 'abcxyq':
    print(f'o[{k}] = {getattr(o, k)}')

# __Base__ objects can also be hashed. Hashing is
# done by using binary provided by pickle
print('---')
print(f'hash(o) = {hash(o)}')

o[a] = 1
o[b] = 2
o[c] = 3
o[x] = a
o[y] = b
o[q] = test
---
o[a] = 1
o[b] = 2
o[c] = 3
o[x] = a
o[y] = b
o[q] = test
---
hash(o) = -2836015521006144951


## Conclusion
Thank you for trying out this tutorial. See the examples in the `examples/` folder for more details on how to use the package. Please checkout the other tutorials in the tutorials folder. Please open an issue here: https://github.com/nasa/hybridq/issues if you run into any issues or have a question. You can also contact Salvatore Mandra at salvatore.madra@nasa.gov.